best possible accuracy

In this lab, you will implement several different classifiers to classify the following datasets:

CIFAR-10, available from tensorflow

Labeled Faces in the Wild (LFW), 

available from Sklearn
CelebA, 

available from Kaggle
For each of the datasets,



*   Preprocessing with principal component analysis 
*   Parameter search
*   Data augmentation (do not use built-in libraries). 

#**CIFAR-10 Tensorflow**

In [ ]:
#Libraries
import time
import math 
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import distutils

from keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.optimizers import *
from keras.utils.vis_utils import plot_model
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

#prepare dataset
def datasetPrepCIFAR(part = 0):
  (x_tr, y_tr), (x_te, y_te) = tf.keras.datasets.cifar10.load_data()

  if (part == 2):   #cnn

    #x_train = x_train.astype('float32') / 255
    #x_test = x_test.astype('float32') / 255
    x_tr = np.float32(x_tr/255)
    x_te = np.float32(x_te/255)

   #convert to one hot rep
    y_tr = tf.keras.utils.to_categorical(y_tr, num_classes=10) 
    y_te = tf.keras.utils.to_categorical(y_te, num_classes=10)
    # Add mirrored images to training set
    x_tr = np.append(x_tr,x_tr[:,:,::-1],axis=0) 
    y_tr = np.append(y_tr,y_tr,axis=0)

  else:
    x_tr = np.float32(x_tr/255).reshape(x_tr.shape[0],-1)
    x_te = np.float32(x_te/255).reshape(x_te.shape[0],-1)

    #classifer
    if(part == 0): 
    
      y_tr = tf.keras.utils.to_categorical(y_tr, num_classes=10)
      y_te = tf.keras.utils.to_categorical(y_te, num_classes=10)
      # Add mirrored images to training set
      x_tr = np.append(x_tr,x_tr[:,::-1],axis=0) 
      y_tr = np.append(y_tr,y_tr,axis=0)
    #svc
    else: 
      y_tr = y_tr.flatten()
      y_te = y_te.flatten()

  return x_tr, y_tr, x_te, y_te

#Decomposition
def ScaleData(mode = 0, prepType = 1):
  x_tr, y_tr, x_te, y_te = datasetPrepCIFAR(prepType)
  
  if (mode == 0):             #default MinMax
    scaler = MinMaxScaler()
    scaler.fit(x_tr)
    x_trainScale = scaler.transform(x_tr)
    x_testScale = scaler.transform(x_te)
    print('Applied MixMax onto the dataset')


  else:                       #Alt: StandardScale
    scaler = StandardScaler()
    scaler.fit(x_tr)
    x_trainScale = scaler.transform(x_tr)
    x_testScale = scaler.transform(x_te)
    print('Applied Standard Scale onto the dataset')

  return x_trainScale, y_tr, x_testScale, y_te

##**Sklearn Classifiers and SVC with CIFAR-10 Dataset(Non-Neural)**

In [ ]:
# Classifer Non-Neural


def evaluate_model(model, x_train, y_train, x_test, y_test):
  m = model()
  m.fit(x_train,y_train)
  return m.score(x_test,y_test)

x_tr, y_tr, x_te, y_te = datasetPrepCIFAR()

# MultinomialNB removed due to Maximum iterations still hasnt converged]
models = [KNeighborsClassifier, RandomForestClassifier,  DecisionTreeClassifier] # These are functions!
model_names = ['K-Nearest Neighbors', 'Random Forest classifer',  'Decision Tree classifer']


acc_list = []
for i in range(len(models)):
  print('Evaluating',model_names[i])
  start = time.time()
  acc_list.append(evaluate_model(models[i], x_tr, y_tr, x_te, y_te))
  done = time.time() - start
  print('Accuracy = {:6.4f}'.format(acc_list[-1]))
  print('Elapsed time = {:.4f} secs'.format(done))

best = np.argmax(acc_list)
print('The best model is',model_names[best])
print('Accuracy = {:6.4f}'.format(acc_list[best]))

170508288/170498071 [==============================] - 3s 0us/step
Evaluating K-Nearest Neighbors
Accuracy = 0.2453
Elapsed time = 2898.6012 secs
Evaluating Random Forest classifer
Accuracy = 0.0793
Elapsed time = 332.2473 secs
Evaluating Decision Tree classifer
Accuracy = 0.2654
Elapsed time = 270.9247 secs
The best model is Decision Tree classifer
Accuracy = 0.2654


In [ ]:
for i in range(0,2):
  x_tr, y_tr, x_te, y_te = ScaleData(mode = i)
  model = SVC()
  model.fit(x_tr, y_tr)
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  pred = model.predict(x_te)
  accuracy = accuracy_score(y_te, pred)
  print(f'Accuracy for SVC with scaling applied to data  {accuracy:.4}')

170508288/170498071 [==============================] - 3s 0us/step
Applied MixMax onto the dataset


Best Non Neural classifer model is svc at Accuracy 44% WITH SCALING DATASET with 1/10 of the dataset since the entire dataset takes over an hour

In [ ]:
x_tr, y_tr, x_te, y_te = datasetPrepCIFAR(1)

for kernel in ['rbf']:
  bestAcc = []
  for i in range(20,71,5):
    pca = PCA(n_components=i, svd_solver="randomized", whiten=True).fit(x_tr)

    x_train_pca = pca.transform(x_tr)
    x_test_pca = pca.transform(x_te)
    
    model = SVC(kernel=kernel, cache_size=1000)
    model.fit(x_train_pca, y_tr)

    pred = model.predict(x_test_pca)
    bestAcc.append(accuracy_score(y_te, pred))
    print(f'Accuracy {bestAcc[-1]:.4}')



  best = np.argmax(bestAcc)
  r = 20 + (5*best)
  print('Best Accuracy range for PCA = ', r, ' using kernel = ', kernel)
  print('Accuracy = {:6.4f}'.format(bestAcc[best]))
    #print(x_train_pca.shape)

Accuracy 0.4952
Accuracy 0.5175
Accuracy 0.5313
Accuracy 0.5354
Accuracy 0.5374


Best Accuracy range for PCA =  70  using kernel =  rbf
Accuracy = 0.5596

In [ ]:
x_tr, y_tr, x_te, y_te = datasetPrepCIFAR(1)
for kernel in ['rbf']:
  bestAcc = []
  for i in range(70,121,5):


    pca = PCA(n_components=i, svd_solver="randomized", whiten=True).fit(x_tr)

    print("Projecting the input data on the eigenfaces orthonormal basis")

    x_train_pca = pca.transform(x_tr)
    x_test_pca = pca.transform(x_te)
    
    model = SVC(kernel=kernel, cache_size=10000)
    model.fit(x_train_pca, y_tr)
    pred = model.predict(x_test_pca)
    bestAcc.append(accuracy_score(y_te, pred))
    print(f'Accuracy {bestAcc[-1]:.4}')

  best = np.argmax(bestAcc)
  r = 70 + (5*best)
  print('Best Accuracy range for PCA = ', r, ' using kernel = ', kernel)
  print('Accuracy = {:6.4f}'.format(bestAcc[best]))
    #print(x_train_pca.shape)

Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.5627
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.5578
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.5605
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.5619
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.5678
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.5683
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.5666
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.5674
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.5668
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.5658
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.5654
Best Accuracy range for PCA =  95  using kernel =  rbf
Accuracy = 0.5683
Projecting the input data on the eigenfaces orthonormal basis
Accurac

Best Accuracy range for PCA =  95  using kernel =  rbf
Accuracy = 0.5683

## **Dense Neural Network Model (MLP) with CIFAR-10 Dataset**

In [ ]:
#To view and see results as graph
def plot_results(all_history):
  loss, val_loss, accuracy, val_accuracy = [], [], [], []
  for history in all_history:
    loss += history.history['loss']
    val_loss += history.history['val_loss']
    accuracy += history.history['accuracy']
    val_accuracy += history.history['val_accuracy']

  fig, ax = plt.subplots()
  ax.plot(accuracy,label = 'train')
  ax.plot(val_accuracy,label = 'test')
  ax.set_title('Accuracy')
  ax.legend(loc='lower right')
  fig, ax = plt.subplots()
  ax.plot(loss,label = 'train')
  ax.plot(val_loss,label = 'test')
  ax.set_title('Loss')
  ax.legend(loc='upper right')

#Set up data and lists for MLP models
x_tr, y_tr, x_te, y_te = datasetPrepCIFAR()


rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=2, verbose=1,min_lr=0.0005)

def dense_model(inputs = 3072,hidden_1=500, hidden_2=500):
  model = tf.keras.models.Sequential()
  model.add(Dense(hidden_1, input_shape=(inputs,), activation='relu'))
  model.add(Dense(hidden_2, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  return model

def dense_model4(inputs = 3072,hidden_1=500, hidden_2=500, hidden_3=500, hidden_4=500):
  model = tf.keras.models.Sequential()
  model.add(Dense(hidden_1, input_shape=(inputs,), activation='relu'))
  model.add(Dense(hidden_2, activation='relu'))
  model.add(Dense(hidden_3, activation='relu'))
  model.add(Dense(hidden_4, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  return model

170508288/170498071 [==============================] - 3s 0us/step


In [ ]:
#Find good range of elearning rate
AccMax_te = []
AccMaxMirror_te = []
LEARN = [.00005,.0001, .0005]

for lr in LEARN:
  dense_network = dense_model4(inputs = x_tr.shape[1])
  dense_network.summary() 
  #learn rate usually at .001
  dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = dense_network.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )
  
  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = dense_network(x_te).numpy()
  # Classify mirrored test set 
  pred2 = dense_network(x_te[:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)
  
  AccMaxMirror_te.append(accuracy_score(pred,y_class))

bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting LEARNING RATE to: {:.4f}'.format(LEARN[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting LEARNING RATE to: {:.4f}'.format(LEARN[bestM]))


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 500)               1536500   
                                                                 
 dense_19 (Dense)            (None, 500)               250500    
                                                                 
 dense_20 (Dense)            (None, 500)               250500    
                                                                 
 dense_21 (Dense)            (None, 500)               250500    
                                                                 
 dense_22 (Dense)            (None, 10)                5010      
                                                                 
Total params: 2,293,010
Trainable params: 2,293,010
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
391/391 [========================

two layers

Highest Mean accuracy acheived on the test set: 0.5078
Highest Mean accuracy acheived by setting LEARNING RATE to: 0.0001
Highest Mean accuracy acheived on the test set with mirror set: 0.5282
Highest Mean accuracy acheived by setting LEARNING RATE to: 0.0005

threelayers: 

Highest Mean accuracy acheived on the test set: 0.5115
Highest Mean accuracy acheived by setting LEARNING RATE to: 0.0001
Highest Mean accuracy acheived on the test set with mirror set: 0.5328
Highest Mean accuracy acheived by setting LEARNING RATE to: 0.0005

 four layers#

Highest Mean accuracy acheived on the test set: 0.5212
Highest Mean accuracy acheived by setting LEARNING RATE to: 0.0001
Highest Mean accuracy acheived on the test set with mirror set: 0.5423
Highest Mean accuracy acheived by setting LEARNING RATE to: 0.0001

five layers:
Highest Mean accuracy acheived on the test set: 0.5128
Highest Mean accuracy acheived by setting LEARNING RATE to: 0.0001
Highest Mean accuracy acheived on the test set with mirror set: 0.5368
Highest Mean accuracy acheived by setting LEARNING RATE to: 0.0001

In [ ]:
#Find good range of batch
AccMax_te = []
AccMaxMirror_te = []
batch = [150, 200, 256, 300] 

for b  in batch:
  dense_network = dense_model4(inputs = x_tr.shape[1])
  dense_network.summary() 
  #learn rate usually at .001
  dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = dense_network.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=b, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )
  
  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = dense_network(x_te).numpy()
  # Classify mirrored test set 
  pred2 = dense_network(x_te[:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))

bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting batch size to: {:.4f}'.format(batch[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting batch size to: {:.4f}'.format(batch[bestM]))

two layers #

Highest Mean accuracy acheived on the test set: 0.5122
Highest Mean accuracy acheived by setting batch size to: 150.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.5231
Highest Mean accuracy acheived by setting batch size to: 200.0000

Four Layers#

Highest Mean accuracy acheived on the test set: 0.5220
Highest Mean accuracy acheived by setting batch size to: 200.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.5438
Highest Mean accuracy acheived by setting batch size to: 256.0000

41%  overall, when changing batch sizes, going lower only increases it so much but it is more about how long it takes to run. focus on speed

Highest Mean accuracy acheived on the test set: 0.4234
Highest Mean accuracy acheived by setting batch to: 300

Highest Mean accuracy acheived on the test set: 0.4278
Highest Mean accuracy acheived by setting batch to: 200

Highest Mean accuracy acheived on the test set: 0.5370
Highest Mean accuracy acheived by setting batch to: 150
entire set


In [ ]:
#Find good range of epo
AccMax_te = []
AccMaxMirror_te = []
epoc = [20, 30, 40, 60, 70]


for e in epoc:
  dense_network = dense_model4(inputs = x_tr.shape[1])
  dense_network.summary() 
  #learn rate usually at .001
  dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = dense_network.fit(
      x_tr, y_tr,
      epochs =  e, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )
  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)
  # Classify test set
  pred1 = dense_network(x_te).numpy()
  # Classify mirrored test set 
  pred2 = dense_network(x_te[:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))

bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting Epoch to: {:.4f}'.format(epoc[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting Epocj to: {:.4f}'.format(epoc[bestM]))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               1536500   
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                                 
 dense_2 (Dense)             (None, 500)               250500    
                                                                 
 dense_3 (Dense)             (None, 500)               250500    
                                                                 
 dense_4 (Dense)             (None, 10)                5010      
                                                                 
Total params: 2,293,010
Trainable params: 2,293,010
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
391/391 [==========================

two layers:

Highest Mean accuracy acheived on the test set: 0.5148
Highest Mean accuracy acheived by setting Epoch to: 60.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.5344
Highest Mean accuracy acheived by setting Epocj to: 70.0000

four layers:
Highest Mean accuracy acheived on the test set: 0.5158
Highest Mean accuracy acheived by setting Epoch to: 30.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.5429
Highest Mean accuracy acheived by setting Epocj to: 60.0000




Highest Mean accuracy acheived on the test set: 0.4412
Highest Mean accuracy acheived by setting epoch to: 100.0000

Highest Mean accuracy acheived on the test set: 0.5487
Highest Mean accuracy acheived by setting epoch to: 60.0000

with learning rate .0001 optimized

In [ ]:
#Find good range of hidden layer
AccMax_te = []
AccMaxMirror_te = []
hid = [250, 500, 1000] 

for h in hid:
  dense_network = dense_model4(inputs = x_tr.shape[1],hidden_1=h, hidden_2=h,hidden_3=h, hidden_4=h)
  dense_network.summary() 
  #learn rate usually at .001
  dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []
  history = dense_network.fit(
    x_tr, y_tr,
    epochs =  60, 
    batch_size=256, 
    verbose = 1,
    validation_data=(x_te, y_te),
    callbacks=[rop]
  )
  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)
  # Classify test set
  pred1 = dense_network(x_te).numpy()
  # Classify mirrored test set 
  pred2 = dense_network(x_te[:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting Hidden Layers to: {:.4f}'.format(hid[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting Hidden Layers to: {:.4f}'.format(hid[bestM]))


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 250)               768250    
                                                                 
 dense_35 (Dense)            (None, 250)               62750     
                                                                 
 dense_36 (Dense)            (None, 250)               62750     
                                                                 
 dense_37 (Dense)            (None, 250)               62750     
                                                                 
 dense_38 (Dense)            (None, 10)                2510      
                                                                 
Total params: 959,010
Trainable params: 959,010
Non-trainable params: 0
_________________________________________________________________
Epoch 1/60
391/391 [============================

two layers#
Highest Mean accuracy acheived on the test set: 0.5293
Highest Mean accuracy acheived by setting Hidden Layers to: 1000.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.5442
Highest Mean accuracy acheived by setting Hidden Layers to: 1000.0000

four layers#
Highest Mean accuracy acheived on the test set: 0.5148
Highest Mean accuracy acheived by setting Hidden Layers to: 1000.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.5404
Highest Mean accuracy acheived by setting Hidden Layers to: 1000.0000

Highest Mean accuracy acheived on the test set: 0.4242
Highest Mean accuracy acheived by setting layers to: 1000.0000

with .0001 learning rate

In [ ]:
dense_network = dense_model4(inputs = x_tr.shape[1], hidden_1=1000, hidden_2=1000, hidden_3=1000, hidden_4=1000 )
dense_network.summary() 

AccMax_te = []
AccMaxMirror_te = []
patiencelist  = [5, 6, 7, 8]

for p in patiencelist:
  rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=p, verbose=1,min_lr=0.0005)

  dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []
  history = dense_network.fit(
    x_tr, y_tr,
    epochs =  60, 
    batch_size=256, 
    verbose = 1,
    validation_data=(x_te, y_te),
    callbacks=[rop]
  )
  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)
  # Classify test set
  pred1 = dense_network(x_te).numpy()
  # Classify mirrored test set 
  pred2 = dense_network(x_te[:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting pateince to: {:.4f}'.format(patiencelist[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting patience to: {:.4f}'.format(patiencelist[bestM]))

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_59 (Dense)            (None, 1000)              3073000   
                                                                 
 dense_60 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_61 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_62 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_63 (Dense)            (None, 10)                10010     
                                                                 
Total params: 6,086,010
Trainable params: 6,086,010
Non-trainable params: 0
_________________________________________________________________
Epoch 1/60
391/391 [=======================

Highest Mean accuracy acheived on the test set: 0.5243
Highest Mean accuracy acheived by setting pateince to: 6.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.5521
Highest Mean accuracy acheived by setting patience to: 6.0000

finally increase by 1% here

In [ ]:
dense_network = dense_model4(inputs = x_tr.shape[1], hidden_1=1000, hidden_2=1000, hidden_3=1000, hidden_4=1000  )
dense_network.summary() 

AccMax_te = []
AccMaxMirror_te = []
min_lrList  = [.001, .005, .01, .05]

for lr in min_lrList :
  rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=6, verbose=1,min_lr=lr)

  dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []
  history = dense_network.fit(
    x_tr, y_tr,
    epochs =  60, 
    batch_size=256, 
    verbose = 1,
    validation_data=(x_te, y_te),
    callbacks=[rop]
  )
  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)
  # Classify test set
  pred1 = dense_network(x_te).numpy()
  # Classify mirrored test set 
  pred2 = dense_network(x_te[:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting Min_LR to: {:.4f}'.format(min_lrList[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting Min_LR to: {:.4f}'.format(min_lrList[bestM]))

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_94 (Dense)            (None, 1000)              3073000   
                                                                 
 dense_95 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_96 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_97 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_98 (Dense)            (None, 10)                10010     
                                                                 
Total params: 6,086,010
Trainable params: 6,086,010
Non-trainable params: 0
_________________________________________________________________
Epoch 1/60
391/391 [=======================

Highest Mean accuracy acheived on the test set: 0.5221
Highest Mean accuracy acheived by setting Min_LR to: 0.0010
Highest Mean accuracy acheived on the test set with mirror set: 0.5446
Highest Mean accuracy acheived by setting Min_LR to: 0.0500

In [ ]:
dense_network = dense_model4(inputs = x_tr.shape[1], hidden_1=1000, hidden_2=1000, hidden_3=1000, hidden_4=1000)
dense_network.summary() 

rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=6, verbose=1,min_lr=.01) #or .05

dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []
history = dense_network.fit(
  x_tr, y_tr,
  epochs =  60, 
  batch_size=256, 
  verbose = 1,
  validation_data=(x_te, y_te),
  callbacks=[rop]
)
all_history.append(history)
AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
y_class = np.argmax(y_te,axis=1)

# Classify test set
pred1 = dense_network(x_te).numpy()
# Classify mirrored test set 
pred2 = dense_network(x_te[:,::-1]).numpy()
pred = np.argmax(pred1+pred2,axis=1)

AccMaxMirror_te.append(accuracy_score(pred,y_class))


## **CNN Model with CIFAR-10 dataset**

In [ ]:
#input_shape= x_tr.shape[1:] OR (32,32,3)  #this is how you get the correct input shape i                                            #into the parameter
# adam find learning rate,  epoch, batch size, 
#input_shape=(32,32,3)  THIS IS the dimension image of cifar-10
#input_shape=(28,28,1) This is the dimension image of MNIST 

def vgg_dropout_bn(input_shape=(32,32,3),blocks=3,branching_factor=2,first_layer_filters=32,dropout=0.2,dense_layer=128):
  model = tf.keras.models.Sequential()
  f = first_layer_filters
  for i in range(blocks):
    if i==0:
      model.add(Conv2D(f, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation="relu"))
    else:
      model.add(Conv2D(f, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(f, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))#add dropout after maxpooling if dropout is implemented
    f = int(f*branching_factor)
  model.add(Flatten())
  if dense_layer>0:
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout)) #add dropout after maxpooling if dropout is implemented
  model.add(Dense(10))
  model.add(Activation('softmax'))
  return model

x_tr, y_tr, x_te, y_te = datasetPrepCIFAR(2)

rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=2, verbose=1,min_lr=0.0005)


170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
#finding best paramter for Numbe rof layers/blocks in the cnn model
AccMax_te = []
AccMaxMirror_te = []

blockList = [4, 5]
for b in blockList:
  cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=b)
  cnn.summary() 
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.025),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = cnn.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting Number of Blocks to: {:.4f}'.format(blockList[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting Number of Blocks to: {:.4f}'.format(blockList[bestM]))


Model: "sequential_84"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_604 (Conv2D)         (None, 32, 32, 32)        896       
                                                                 
 conv2d_605 (Conv2D)         (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_301 (MaxPooli  (None, 16, 16, 32)       0         
 ng2D)                                                           
                                                                 
 batch_normalization_374 (Ba  (None, 16, 16, 32)       128       
 tchNormalization)                                               
                                                                 
 dropout_374 (Dropout)       (None, 16, 16, 32)        0         
                                                                 
 conv2d_606 (Conv2D)         (None, 16, 16, 64)      

Max layers to be added is 5 before unable to add anymore layers

Highest Mean accuracy acheived on the test set: 0.8850
Highest Mean accuracy acheived by setting Number of Blocks to: 5.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.8927
Highest Mean accuracy acheived by setting Number of Blocks to: 5.0000

In [ ]:
#Find good range of elearning rate for adam
AccMax_te = []
AccMaxMirror_te = []
LEARN = [.005, .0075, .01, .025]

for lr in LEARN:

  cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=5)  
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = cnn.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting Min_LR to: {:.4f}'.format(LEARN[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting Min_LR to: {:.4f}'.format(LEARN[bestM]))

Epoch 1/30
391/391 [==============================] - 13s 28ms/step - loss: 1.4808 - accuracy: 0.4623 - val_loss: 1.2993 - val_accuracy: 0.5749 - lr: 0.0050
Epoch 2/30
391/391 [==============================] - 10s 26ms/step - loss: 0.8621 - accuracy: 0.6992 - val_loss: 1.1090 - val_accuracy: 0.6637 - lr: 0.0050
Epoch 3/30
391/391 [==============================] - 10s 26ms/step - loss: 0.6798 - accuracy: 0.7653 - val_loss: 1.0178 - val_accuracy: 0.6727 - lr: 0.0050
Epoch 4/30
391/391 [==============================] - 10s 26ms/step - loss: 0.5785 - accuracy: 0.8017 - val_loss: 0.6820 - val_accuracy: 0.7728 - lr: 0.0050
Epoch 5/30
391/391 [==============================] - 10s 26ms/step - loss: 0.5104 - accuracy: 0.8246 - val_loss: 1.2390 - val_accuracy: 0.6505 - lr: 0.0050
Epoch 6/30
391/391 [==============================] - 10s 26ms/step - loss: 0.4582 - accuracy: 0.8422 - val_loss: 0.6743 - val_accuracy: 0.7888 - lr: 0.0050
Epoch 7/30
391/391 [==============================] - 10s 

ighest Mean accuracy acheived on the test set: 0.8844
Highest Mean accuracy acheived by setting Min_LR to: 0.0100
Highest Mean accuracy acheived on the test set with mirror set: 0.8960
Highest Mean accuracy acheived by setting Min_LR to: 0.0100

Highest Mean accuracy acheived on the test set: 0.8813
Highest Mean accuracy acheived by setting Min_LR to: 0.0050
Highest Mean accuracy acheived on the test set with mirror set: 0.8913
Highest Mean accuracy acheived by setting Min_LR to: 0.0050

Highest Mean accuracy acheived on the test set: 0.8801
Highest Mean accuracy acheived by setting Min_LR to: 0.0100
Highest Mean accuracy acheived on the test set with mirror set: 0.8891
Highest Mean accuracy acheived by setting Min_LR to: 0.0100

Highest Mean accuracy acheived on the test set: 0.8817
Highest Mean accuracy acheived by setting Min_LR to: 0.0100
Highest Mean accuracy acheived on the test set with mirror set: 0.8904
Highest Mean accuracy acheived by setting Min_LR to: 0.0250

In [ ]:
#Find good range for dropout 
AccMax_te = []
AccMaxMirror_te = []
drop = [.15, .2, .3, .4]

for d in drop:

  cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=5, dropout = d)  
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.025),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = cnn.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting dropout to: {:.4f}'.format(drop[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting drpout to: {:.4f}'.format(drop[bestM]))

Epoch 1/30
391/391 [==============================] - 12s 27ms/step - loss: 1.7705 - accuracy: 0.3413 - val_loss: 1.8402 - val_accuracy: 0.3683 - lr: 0.0250
Epoch 2/30
391/391 [==============================] - 10s 26ms/step - loss: 1.1249 - accuracy: 0.5945 - val_loss: 1.4980 - val_accuracy: 0.5266 - lr: 0.0250
Epoch 3/30
391/391 [==============================] - 10s 26ms/step - loss: 0.8409 - accuracy: 0.7059 - val_loss: 1.0325 - val_accuracy: 0.6786 - lr: 0.0250
Epoch 4/30
391/391 [==============================] - 10s 26ms/step - loss: 0.6935 - accuracy: 0.7621 - val_loss: 0.7696 - val_accuracy: 0.7499 - lr: 0.0250
Epoch 5/30
391/391 [==============================] - 10s 26ms/step - loss: 0.5999 - accuracy: 0.7946 - val_loss: 0.6968 - val_accuracy: 0.7730 - lr: 0.0250
Epoch 6/30
391/391 [==============================] - 10s 26ms/step - loss: 0.5282 - accuracy: 0.8191 - val_loss: 0.7628 - val_accuracy: 0.7714 - lr: 0.0250
Epoch 7/30
391/391 [==============================] - 10s 

Highest Mean accuracy acheived on the test set: 0.8811
Highest Mean accuracy acheived by setting dropout to: 0.4000
Highest Mean accuracy acheived on the test set with mirror set: 0.8913
Highest Mean accuracy acheived by setting drpout to: 0.1000

In [ ]:
#Find good range of for batchsize
AccMax_te = []
AccMaxMirror_te = []

batchS = [150, 200, 256, 350]

cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=5, dropout =.1)  
cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= .01),loss="categorical_crossentropy", metrics=["accuracy"])

for b in batchS:
  all_history = []
  history = cnn.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=b, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting Batch size to: {:.4f}'.format(batchS[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting batch size to: {:.4f}'.format(batchS[bestM]))

Epoch 1/30
667/667 [==============================] - 14s 19ms/step - loss: 1.5202 - accuracy: 0.4359 - val_loss: 1.4349 - val_accuracy: 0.5450 - lr: 0.0100
Epoch 2/30
667/667 [==============================] - 12s 18ms/step - loss: 0.8991 - accuracy: 0.6823 - val_loss: 1.0028 - val_accuracy: 0.6634 - lr: 0.0100
Epoch 3/30
667/667 [==============================] - 12s 18ms/step - loss: 0.6778 - accuracy: 0.7663 - val_loss: 0.7764 - val_accuracy: 0.7499 - lr: 0.0100
Epoch 4/30
667/667 [==============================] - 12s 18ms/step - loss: 0.5683 - accuracy: 0.8057 - val_loss: 0.6522 - val_accuracy: 0.7798 - lr: 0.0100
Epoch 5/30
667/667 [==============================] - 12s 18ms/step - loss: 0.4876 - accuracy: 0.8339 - val_loss: 0.6147 - val_accuracy: 0.7945 - lr: 0.0100
Epoch 6/30
667/667 [==============================] - 12s 18ms/step - loss: 0.4347 - accuracy: 0.8507 - val_loss: 0.6238 - val_accuracy: 0.7972 - lr: 0.0100
Epoch 7/30
667/667 [==============================] - 12s 

Highest Mean accuracy acheived on the test set: 0.8831
Highest Mean accuracy acheived by setting Batch size to: 350.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.8961
Highest Mean accuracy acheived by setting batch size to: 256.0000

In [ ]:
#Find good range of for epochs
AccMax_te = []
AccMaxMirror_te = []

epoc = [30, 40, 50]

cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=5, dropout = .1)  
cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= .01),loss="categorical_crossentropy", metrics=["accuracy"])

for ep in epoc:
  all_history = []
  history = cnn.fit(
      x_tr, y_tr,
      epochs =  ep, 
      batch_size= 256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting Epoch to: {:.4f}'.format(epoc[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting Epoch to: {:.4f}'.format(epoc[bestM]))

Epoch 1/30
391/391 [==============================] - 29s 29ms/step - loss: 1.6105 - accuracy: 0.4026 - val_loss: 1.5449 - val_accuracy: 0.4646 - lr: 0.0100
Epoch 2/30
391/391 [==============================] - 10s 26ms/step - loss: 0.9507 - accuracy: 0.6622 - val_loss: 1.2071 - val_accuracy: 0.6328 - lr: 0.0100
Epoch 3/30
391/391 [==============================] - 10s 26ms/step - loss: 0.6998 - accuracy: 0.7577 - val_loss: 0.9036 - val_accuracy: 0.7058 - lr: 0.0100
Epoch 4/30
391/391 [==============================] - 10s 26ms/step - loss: 0.5748 - accuracy: 0.8012 - val_loss: 0.9368 - val_accuracy: 0.6934 - lr: 0.0100
Epoch 5/30
391/391 [==============================] - 10s 26ms/step - loss: 0.4945 - accuracy: 0.8295 - val_loss: 0.7429 - val_accuracy: 0.7626 - lr: 0.0100
Epoch 6/30
391/391 [==============================] - 10s 26ms/step - loss: 0.4336 - accuracy: 0.8502 - val_loss: 0.5942 - val_accuracy: 0.7999 - lr: 0.0100
Epoch 7/30
391/391 [==============================] - 10s 

Highest Mean accuracy acheived on the test set: 0.8726
Highest Mean accuracy acheived by setting Epoch to: 50.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.8864
Highest Mean accuracy acheived by setting Epoch to: 40.0000

In [ ]:
#Find good range of min_LR
AccMax_te = []
AccMaxMirror_te = []
min_lrList  = [.000005, .00001, .00005]
#.0001, .0005, .001, .005, .01, .05,

for lr in min_lrList :
  rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=6, verbose=1,min_lr=lr)
  cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=5, dropout = .1)  
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= .01),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = cnn.fit(
      x_tr, y_tr,
      epochs =  40, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting Epoch to: {:.6f}'.format(min_lrList[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting Epoch to: {:.6f}'.format(min_lrList[bestM]))

Epoch 1/40
391/391 [==============================] - 12s 27ms/step - loss: 1.6512 - accuracy: 0.3930 - val_loss: 2.1181 - val_accuracy: 0.3882 - lr: 0.0100
Epoch 2/40
391/391 [==============================] - 10s 26ms/step - loss: 0.9940 - accuracy: 0.6432 - val_loss: 1.3874 - val_accuracy: 0.5586 - lr: 0.0100
Epoch 3/40
391/391 [==============================] - 10s 26ms/step - loss: 0.7383 - accuracy: 0.7426 - val_loss: 0.9628 - val_accuracy: 0.7040 - lr: 0.0100
Epoch 4/40
391/391 [==============================] - 10s 26ms/step - loss: 0.6058 - accuracy: 0.7897 - val_loss: 0.7265 - val_accuracy: 0.7695 - lr: 0.0100
Epoch 5/40
391/391 [==============================] - 10s 26ms/step - loss: 0.5172 - accuracy: 0.8226 - val_loss: 0.6975 - val_accuracy: 0.7704 - lr: 0.0100
Epoch 6/40
391/391 [==============================] - 10s 26ms/step - loss: 0.4565 - accuracy: 0.8421 - val_loss: 0.7872 - val_accuracy: 0.7594 - lr: 0.0100
Epoch 7/40
391/391 [==============================] - 10s 

Highest Mean accuracy acheived on the test set: 0.8379
Highest Mean accuracy acheived by setting Epoch to: 0.1000
Highest Mean accuracy acheived on the test set with mirror set: 0.8664
Highest Mean accuracy acheived by setting Epoch to: 0.5000

Highest Mean accuracy acheived on the test set: 0.8781
Highest Mean accuracy acheived by setting Epoch to: 0.0005
Highest Mean accuracy acheived on the test set with mirror set: 0.8923
Highest Mean accuracy acheived by setting Epoch to: 0.0001

In [ ]:
#Best model acheive


rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=6, verbose=1,min_lr=0.00001)
cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=5, dropout = .1)  
cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= .01),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

history = cnn.fit(
    x_tr, y_tr,
    epochs =  40, 
    batch_size=256, 
    verbose = 1,
    validation_data=(x_te, y_te),
    callbacks=[rop]
)

all_history.append(history)
y_class = np.argmax(y_te,axis=1)

# Classify test set
pred1 = cnn(x_te).numpy()
# Classify mirrored test set 
pred2 = cnn(x_te[:,:,::-1]).numpy()
pred = np.argmax(pred1+pred2,axis=1)



print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(accuracy_score(pred,y_class)))

Epoch 1/40
391/391 [==============================] - 12s 27ms/step - loss: 1.5774 - accuracy: 0.4186 - val_loss: 1.9448 - val_accuracy: 0.3974 - lr: 0.0100
Epoch 2/40
391/391 [==============================] - 10s 26ms/step - loss: 0.9524 - accuracy: 0.6643 - val_loss: 1.1721 - val_accuracy: 0.6318 - lr: 0.0100
Epoch 3/40
391/391 [==============================] - 10s 26ms/step - loss: 0.7247 - accuracy: 0.7485 - val_loss: 0.8719 - val_accuracy: 0.7228 - lr: 0.0100
Epoch 4/40
391/391 [==============================] - 10s 26ms/step - loss: 0.5989 - accuracy: 0.7945 - val_loss: 0.8633 - val_accuracy: 0.7341 - lr: 0.0100
Epoch 5/40
391/391 [==============================] - 10s 26ms/step - loss: 0.5120 - accuracy: 0.8230 - val_loss: 0.8932 - val_accuracy: 0.7180 - lr: 0.0100
Epoch 6/40
391/391 [==============================] - 10s 26ms/step - loss: 0.4513 - accuracy: 0.8444 - val_loss: 0.6775 - val_accuracy: 0.7988 - lr: 0.0100
Epoch 7/40
391/391 [==============================] - 10s 

#**Sklearn LFW dataset**

In [ ]:
#Libraries
import time
import math 
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import distutils

from keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.optimizers import *
from keras.utils.vis_utils import plot_model
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,mean_squared_error,mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_lfw_people

#prepare dataset
def datasetPrepLFW(mode = 0, subset = .25, MinFace = 50):
  
  lfw_people = fetch_lfw_people(min_faces_per_person=MinFace, resize=0.5)
  x = lfw_people.data 

  # the label to predict is the id of the person
  y = lfw_people.target
  target_names = lfw_people.target_names
  n_classes = target_names.shape[0]

  #cnn
  if (mode == 2): 
    images = lfw_people.images
    yHot = tf.keras.utils.to_categorical(y, num_classes=n_classes) 

    x_tr, x_te, y_tr, y_te = train_test_split(images, yHot, test_size=subset, random_state=42)
    # add empty color dimension and scale to [0,1]
    x_tr = np.float32(np.expand_dims(x_tr, -1)/255)
    x_te = np.float32(np.expand_dims(x_te, -1)/255)

    # Add mirrored images to training set
    x_tr = np.append(x_tr,x_tr[:,:,::-1],axis=0) 
    y_tr = np.append(y_tr,y_tr,axis=0)

  else:
    x_tr, x_te, y_tr, y_te = train_test_split(x, y, test_size=subset, random_state=42)
    #reshape
    #x_tr = np.float32(x_tr/255).reshape(x_tr.shape[0],-1)
    #x_te = np.float32(x_te/255).reshape(x_te.shape[0],-1)

    #classifer
    if(mode == 0): 
      y_tr = tf.keras.utils.to_categorical(y_tr, num_classes=n_classes)
      y_te = tf.keras.utils.to_categorical(y_te, num_classes=n_classes)


      x_tr = np.append(x_tr,x_tr[:,::-1],axis=0) 
      y_tr = np.append(y_tr,y_tr,axis=0)
    #svc
    else: 
      y_tr = y_tr.flatten()
      y_te = y_te.flatten()
        # Add mirrored images to training set

  return x_tr, y_tr, x_te, y_te



def ScaleData(mode = 0, prepType = 1):
  x_tr, y_tr, x_te, y_te = datasetPrepLFW(prepType)
  
  if (mode == 0):             #default MinMax
    scaler = MinMaxScaler()
    scaler.fit(x_tr)
    x_trainScale = scaler.transform(x_tr)
    x_testScale = scaler.transform(x_te)
    print('Applied MixMax onto the dataset')


  else:                       #Alt: StandardScale
    scaler = StandardScaler()
    scaler.fit(x_tr)
    x_trainScale = scaler.transform(x_tr)
    x_testScale = scaler.transform(x_te)
    print('Applied Standard Scale onto the dataset')

  return x_trainScale, y_tr, x_testScale, y_te

##**Sklearn Classifers and SVC with LFW dataset(Non-neural)**

In [ ]:
# Classifer Sklearn

x_tr, y_tr, x_te, y_te = datasetPrepLFW()

def evaluate_model(model, x_train, y_train, x_test, y_test):
  m = model()
  m.fit(x_train,y_train)
  return m.score(x_test,y_test)


# MultinomialNB removed due to Maximum iterations still hasnt converged
models = [KNeighborsClassifier, RandomForestClassifier,  DecisionTreeClassifier] # These are functions!
model_names = ['K-Nearest Neighbors', 'Random Forest classifer',  'Decision Tree classifer']


acc_list = []
for i in range(len(models)):
  print('Evaluating',model_names[i])
  start = time.time()
  acc_list.append(evaluate_model(models[i], x_tr, y_tr, x_te, y_te))
  done = time.time() - start
  print('Accuracy = {:6.4f}'.format(acc_list[-1]))
  print('Elapsed time = {:.4f} secs'.format(done))

best = np.argmax(acc_list)
print('The best model is',model_names[best])
print('Accuracy = {:6.4f}'.format(acc_list[best]))

Evaluating K-Nearest Neighbors
Accuracy = 0.3846
Elapsed time = 5.9242 secs
Evaluating Random Forest classifer
Accuracy = 0.1692
Elapsed time = 13.5661 secs
Evaluating Decision Tree classifer
Accuracy = 0.3154
Elapsed time = 10.7016 secs
The best model is K-Nearest Neighbors
Accuracy = 0.3846


In [ ]:
#SVC EDIT FOR LFW!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# DO IT NOW!!!! EDIT CHANGE IT ALTER IT FROM CIFAR

#SVC
x_tr, y_tr, x_te, y_te = datasetPrepLFW(1)

C = [.1, 1.0, 5.0]
Gamma = [.1, .5, 1.0]
AccuracyList = []
SVCmodel = SVC()

for kernel in ['rbf', 'poly']:
  for c in C:
    for G in Gamma:
      SVCmodel = SVC(C = c, kernel=kernel, gamma = G, cache_size=1000, coef0 = 0.5)
      SVCmodel.fit(x_tr, y_tr)
      pred = SVCmodel.predict(x_te)
      AccuracyList.append(accuracy_score(y_te, pred))
      print(f"SVC Model with ={kernel}, C={c}, Gamma ={G}")
      print()

def Combo(Position):
  if(Position <9):
    m = 'rbf'
  else:
    m = 'poly'
    Position = Position - 9

  c = C[math.floor(Position/3)]
  g= Gamma[Position%3]
  return m, c, g

bestA = np.argmax(AccuracyList)
Am, Ac, Ag = Combo(bestA)
print(f'Best SVC combo for Accuracy is kernal={Am}, C={Ac}, Gamma={Ag}')
print('Accuracy = {:6.4f}'.format(AccuracyList[bestA]))


SVC Model with =rbf, C=0.1, Gamma =0.1

SVC Model with =rbf, C=0.1, Gamma =0.5

SVC Model with =rbf, C=0.1, Gamma =1.0

SVC Model with =rbf, C=1.0, Gamma =0.1

SVC Model with =rbf, C=1.0, Gamma =0.5

SVC Model with =rbf, C=1.0, Gamma =1.0

SVC Model with =rbf, C=5.0, Gamma =0.1

SVC Model with =rbf, C=5.0, Gamma =0.5

SVC Model with =rbf, C=5.0, Gamma =1.0

SVC Model with =poly, C=0.1, Gamma =0.1

SVC Model with =poly, C=0.1, Gamma =0.5

SVC Model with =poly, C=0.1, Gamma =1.0

SVC Model with =poly, C=1.0, Gamma =0.1

SVC Model with =poly, C=1.0, Gamma =0.5

SVC Model with =poly, C=1.0, Gamma =1.0

SVC Model with =poly, C=5.0, Gamma =0.1

SVC Model with =poly, C=5.0, Gamma =0.5

SVC Model with =poly, C=5.0, Gamma =1.0

Best SVC combo for Accuracy is kernal=poly, C=0.1, Gamma=0.1
Accuracy = 0.7154


Best SVC combo for Accuracy is kernal=poly, C=0.1, Gamma=0.1
Accuracy = 0.7154


In [ ]:
x_tr, y_tr, x_te, y_te = datasetPrepLFW(1)

for kernel in ['rbf']:
  bestAcc = []
  AccMaxMirror_te = []

  for i in range(40,66,5):

    pca = PCA(n_components=i, svd_solver="randomized", whiten=True).fit(x_tr)

    x_train_pca = pca.transform(x_tr)
    x_test_pca = pca.transform(x_te)
    
    model = SVC(kernel=kernel, cache_size=10000)
    model.fit(x_train_pca, y_tr)
    pred = model.predict(x_test_pca)
    bestAcc.append(accuracy_score(y_te, pred))
    print(f'Accuracy {bestAcc[-1]:.4}')


best = np.argmax(bestAcc)
r = 40 + (5*best)
print('Best Accuracy range for PCA = ', r, ' using kernel = ', kernel)
print('Accuracy = {:6.4f}'.format(bestAcc[best]))

Accuracy 0.7897
Accuracy 0.7974
Accuracy 0.8103
Accuracy 0.8103
Accuracy 0.8128
Accuracy 0.8154
Best Accuracy range for PCA =  65  using kernel =  rbf
Accuracy = 0.8154


In [ ]:
x_tr, y_tr, x_te, y_te = datasetPrepLFW(1)

for kernel in ['rbf']:
  bestAcc = []
  for i in range(70,100,5):
    pca = PCA(n_components=i, svd_solver="randomized", whiten=True).fit(x_tr)

    print("Projecting the input data on the eigenfaces orthonormal basis")

    x_train_pca = pca.transform(x_tr)
    x_test_pca = pca.transform(x_te)
    
    model = SVC(kernel=kernel, cache_size=10000)
    model.fit(x_train_pca, y_tr)
    pred = model.predict(x_test_pca)
    bestAcc.append(accuracy_score(y_te, pred))
    print(f'Accuracy {bestAcc[-1]:.4}')

  best = np.argmax(bestAcc)
  r = 70 + (5*best)
  print('Best Accuracy range for PCA = ', r, ' using kernel = ', kernel)
  print('Accuracy = {:6.4f}'.format(bestAcc[best]))
#print(X_train_pca.shape)

Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.8231
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.8385
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.8282
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.8179
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.8205
Projecting the input data on the eigenfaces orthonormal basis
Accuracy 0.8077
Best Accuracy range for PCA =  75  using kernel =  rbf
Accuracy = 0.8385


Best Non neural model. SVC with PCA set to 80 for 83% accuracy

## **Dense Neural Network Model(MLP) with LFW dataset**

In [ ]:
#View graph of the results 
def plot_results(all_history):
  loss, val_loss, accuracy, val_accuracy = [], [], [], []
  for history in all_history:
    loss += history.history['loss']
    val_loss += history.history['val_loss']
    accuracy += history.history['accuracy']
    val_accuracy += history.history['val_accuracy']

  fig, ax = plt.subplots()
  ax.plot(accuracy,label = 'train')
  ax.plot(val_accuracy,label = 'test')
  ax.set_title('Accuracy')
  ax.legend(loc='lower right')
  fig, ax = plt.subplots()
  ax.plot(loss,label = 'train')
  ax.plot(val_loss,label = 'test')
  ax.set_title('Loss')
  ax.legend(loc='upper right')

#Set up data and lists for MLP models
x_tr, y_tr, x_te, y_te = datasetPrepLFW()
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=2, verbose=1,min_lr=0.0005)

def dense_model3(inputs = 2914,hidden_1=500, hidden_2=500, hidden_3=500):
  model = tf.keras.models.Sequential()
  model.add(Dense(hidden_1, input_shape=(inputs,), activation='relu'))
  model.add(Dense(hidden_2, activation='relu'))
  model.add(Dense(hidden_3, activation='relu'))
  model.add(Dense(12, activation='softmax'))
  return model
def dense_model4(inputs = 2914,hidden_1=500, hidden_2=500, hidden_3=500, hidden_4=500):
  model = tf.keras.models.Sequential()
  model.add(Dense(hidden_1, input_shape=(inputs,), activation='relu'))
  model.add(Dense(hidden_2, activation='relu'))
  model.add(Dense(hidden_3, activation='relu'))
  model.add(Dense(hidden_4, activation='relu'))
  model.add(Dense(12, activation='softmax'))
  return model

In [ ]:
#Find good range of elearning rate
AccMax_te = []
AccMaxMirror_te = []
LEARN = [.00005, .0001, .0005, .001]

for lr in LEARN:
  dense_network = dense_model4(inputs = x_tr.shape[1])
  dense_network.summary() 
  #learn rate usually at .001
  rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=2, verbose=1,min_lr=lr)

  dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0001),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = dense_network.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )
  
  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = dense_network(x_te).numpy()
  # Classify mirrored test set 
  pred2 = dense_network(x_te[:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)
  
  AccMaxMirror_te.append(accuracy_score(pred,y_class))

bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting min_LEARNING RATE to: {:.4f}'.format(LEARN[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting min_LEARNING RATE to: {:.4f}'.format(LEARN[bestM]))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               1457500   
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                                 
 dense_2 (Dense)             (None, 500)               250500    
                                                                 
 dense_3 (Dense)             (None, 500)               250500    
                                                                 
 dense_4 (Dense)             (None, 12)                6012      
                                                                 
Total params: 2,215,012
Trainable params: 2,215,012
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
10/10 [============================

Highest Mean accuracy acheived on the test set: 0.5256
Highest Mean accuracy acheived by setting min_LEARNING RATE to: 0.0001
Highest Mean accuracy acheived on the test set with mirror set: 0.6051
Highest Mean accuracy acheived by setting min_LEARNING RATE to: 0.0001

In [ ]:
#Find good range of elearning rate
AccMax_te = []
AccMaxMirror_te = []
patienc = [4, 5, 6, 7]

for p in patienc:
  dense_network = dense_model4(inputs = x_tr.shape[1])
  dense_network.summary() 
  #learn rate usually at .001
  rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=p, verbose=1,min_lr=0.0001)

  dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0001),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = dense_network.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )
  
  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = dense_network(x_te).numpy()
  # Classify mirrored test set 
  pred2 = dense_network(x_te[:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)
  
  AccMaxMirror_te.append(accuracy_score(pred,y_class))

bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting patience to: {:.4f}'.format(patienc[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting patience to: {:.4f}'.format(patienc[bestM]))


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 500)               1457500   
                                                                 
 dense_21 (Dense)            (None, 500)               250500    
                                                                 
 dense_22 (Dense)            (None, 500)               250500    
                                                                 
 dense_23 (Dense)            (None, 500)               250500    
                                                                 
 dense_24 (Dense)            (None, 12)                6012      
                                                                 
Total params: 2,215,012
Trainable params: 2,215,012
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
10/10 [==========================

Highest Mean accuracy acheived on the test set: 0.5031
Highest Mean accuracy acheived by setting patience to: 4.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.5462
Highest Mean accuracy acheived by setting patience to: 8.0000

3 layers

Highest Mean accuracy acheived on the test set: 0.4662
Highest Mean accuracy acheived by setting LEARNING RATE to: 0.0005
Highest Mean accuracy acheived on the test set with mirror set: 0.5795
Highest Mean accuracy acheived by setting LEARNING RATE to: 0.0001

In [ ]:
#Find good range of elearning rate
AccMax_te = []
AccMaxMirror_te = []
hid= [3000, 3500, 4000, 5000, 6000]
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=4, verbose=1,min_lr=0.0001)

for h in hid:
  dense_network = dense_model4(inputs = x_tr.shape[1], hidden_1 = h, hidden_2 = h, hidden_3= h, hidden_4= h)
  dense_network.summary() 
  #learn rate usually at .001
  dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0001),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = dense_network.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )
  
  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = dense_network(x_te).numpy()
  # Classify mirrored test set 
  pred2 = dense_network(x_te[:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)
  
  AccMaxMirror_te.append(accuracy_score(pred,y_class))

bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting hidden layers to: {:.4f}'.format(hid[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting hidden layers to: {:.4f}'.format(hid[bestM]))


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 3000)              8745000   
                                                                 
 dense_41 (Dense)            (None, 3000)              9003000   
                                                                 
 dense_42 (Dense)            (None, 3000)              9003000   
                                                                 
 dense_43 (Dense)            (None, 3000)              9003000   
                                                                 
 dense_44 (Dense)            (None, 12)                36012     
                                                                 
Total params: 35,790,012
Trainable params: 35,790,012
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
10/10 [========================




four layers
Highest Mean accuracy acheived on the test set with mirror set: 0.7154
Highest Mean accuracy acheived by setting hidden layers to: 1500.0000

Highest Mean accuracy acheived on the test set with mirror set: 0.7231
Highest Mean accuracy acheived by setting hidden layers to: 2000.0000

Highest Mean accuracy acheived on the test set: 0.7400
Highest Mean accuracy acheived by setting hidden layers to: 3000.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.7821
Highest Mean accuracy acheived by setting hidden layers to: 3000.0000

Highest Mean accuracy acheived on the test set: 0.7769
Highest Mean accuracy acheived by setting hidden layers to: 6000.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.8179
Highest Mean accuracy acheived by setting hidden layers to: 6000.0000

In [ ]:
#Find good range of elearning rate
AccMax_te = []
AccMaxMirror_te = []
batc= [256, 275, 300]
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=4, verbose=1,min_lr=0.0001)

for b in batc:
  dense_network = dense_model4(inputs = x_tr.shape[1], hidden_1 = 6000, hidden_2 = 6000, hidden_3= 6000, hidden_4= 6000)
  dense_network.summary() 
  #learn rate usually at .001
  dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0001),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = dense_network.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=b, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )
  
  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = dense_network(x_te).numpy()
  # Classify mirrored test set 
  pred2 = dense_network(x_te[:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)
  
  AccMaxMirror_te.append(accuracy_score(pred,y_class))

bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting batch size to: {:.4f}'.format(batc[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting batch size to: {:.4f}'.format(batc[bestM]))


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_65 (Dense)            (None, 6000)              17490000  
                                                                 
 dense_66 (Dense)            (None, 6000)              36006000  
                                                                 
 dense_67 (Dense)            (None, 6000)              36006000  
                                                                 
 dense_68 (Dense)            (None, 6000)              36006000  
                                                                 
 dense_69 (Dense)            (None, 12)                72012     
                                                                 
Total params: 125,580,012
Trainable params: 125,580,012
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
10/10 [=====================

Highest Mean accuracy acheived on the test set: 0.7856
Highest Mean accuracy acheived by setting batch size to: 275.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.8103
Highest Mean accuracy acheived by setting batch size to: 275.0000
Highest Mean accuracy acheived on the test set: 0.7903
Highest Mean accuracy acheived by setting batch size to: 275.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.8077
Highest Mean accuracy acheived by setting batch size to: 300.0000

In [ ]:
#Find good range of elearning rate
AccMax_te = []
AccMaxMirror_te = []
epoc= [30, 40, 50, 60]
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=4, verbose=1,min_lr=0.0001)

for ep in epoc:
  dense_network = dense_model4(inputs = x_tr.shape[1], hidden_1 = 6000, hidden_2 = 6000, hidden_3= 6000, hidden_4= 6000)
  dense_network.summary() 
  #learn rate usually at .001
  dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0001),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = dense_network.fit(
      x_tr, y_tr,
      epochs =  ep, 
      batch_size=300, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )
  
  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = dense_network(x_te).numpy()
  # Classify mirrored test set 
  pred2 = dense_network(x_te[:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)
  
  AccMaxMirror_te.append(accuracy_score(pred,y_class))

bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting epoch to: {:.4f}'.format(epoc[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting epoch to: {:.4f}'.format(epoc[bestM]))


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_80 (Dense)            (None, 6000)              17490000  
                                                                 
 dense_81 (Dense)            (None, 6000)              36006000  
                                                                 
 dense_82 (Dense)            (None, 6000)              36006000  
                                                                 
 dense_83 (Dense)            (None, 6000)              36006000  
                                                                 
 dense_84 (Dense)            (None, 12)                72012     
                                                                 
Total params: 125,580,012
Trainable params: 125,580,012
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
8/8 [=======================

Highest Mean accuracy acheived on the test set: 0.7918
Highest Mean accuracy acheived by setting epoch to: 60.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.7974
Highest Mean accuracy acheived by setting epoch to: 30.0000

In [ ]:
#Best Dense model

rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=4, verbose=1,min_lr=0.0001)

dense_network = dense_model4(inputs = x_tr.shape[1], hidden_1 = 6000, hidden_2 = 6000, hidden_3= 6000, hidden_4= 6000)
dense_network.summary() 
#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0001),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

history = dense_network.fit(
    x_tr, y_tr,
    epochs =  30, 
    batch_size=300, 
    verbose = 1,
    validation_data=(x_te, y_te),
    callbacks=[rop]
)

all_history.append(history)

y_class = np.argmax(y_te,axis=1)

# Classify test set
pred1 = dense_network(x_te).numpy()
# Classify mirrored test set 
pred2 = dense_network(x_te[:,::-1]).numpy()
pred = np.argmax(pred1+pred2,axis=1)



print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(accuracy_score(pred,y_class)))

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_100 (Dense)           (None, 6000)              17490000  
                                                                 
 dense_101 (Dense)           (None, 6000)              36006000  
                                                                 
 dense_102 (Dense)           (None, 6000)              36006000  
                                                                 
 dense_103 (Dense)           (None, 6000)              36006000  
                                                                 
 dense_104 (Dense)           (None, 12)                72012     
                                                                 
Total params: 125,580,012
Trainable params: 125,580,012
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
8/8 [=======================

ighest Mean accuracy acheived on the test set: 0.7769
Highest Mean accuracy acheived on the test set with mirror set: 0.8026

##**CNN Model with LFW**

In [ ]:
#input_shape=(32,32,3)  THIS IS the dimension image of cifar-10
#input_shape=(28,28,1) This is the dimension image of MNIST 


def vgg_dropout_bn(input_shape=(32,32,3),blocks=3,branching_factor=2,first_layer_filters=32,dropout=0.2,dense_layer=128):
  model = tf.keras.models.Sequential()
  f = first_layer_filters
  for i in range(blocks):
    if i==0:
      model.add(Conv2D(f, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation="relu"))
    else:
      model.add(Conv2D(f, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(f, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))#add dropout after maxpooling if dropout is implemented
    f = int(f*branching_factor)
  model.add(Flatten())
  if dense_layer>0:
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout)) #add dropout after maxpooling if dropout is implemented
  model.add(Dense(12))
  model.add(Activation('softmax'))
  return model

x_tr, y_tr, x_te, y_te = datasetPrepLFW(2)
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=2, verbose=1,min_lr=0.0005)

In [ ]:
#finding best paramter for Numbe rof layers/blocks in the cnn model
AccMax_te = []
AccMaxMirror_te = []

blockList = [3, 4, 5]
for b in blockList:
  cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=b)
  cnn.summary() 
  #0.0075
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.0075),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = cnn.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting Number of Blocks to: {:.4f}'.format(blockList[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting Number of Blocks to: {:.4f}'.format(blockList[bestM]))


Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 47, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 47, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 23, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 31, 23, 32)       128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 31, 23, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 31, 23, 64)      

Highest Mean accuracy acheived on the test set: 0.8651
Highest Mean accuracy acheived by setting Number of Blocks to: 4.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.8872
Highest Mean accuracy acheived by setting Number of Blocks to: 4.0000

In [ ]:
#Find good range of elearning rate for adam
AccMax_te = []
AccMaxMirror_te = []
LEARN = [.005, .0075, .01, .015]

for lr in LEARN:
  cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=4)
  cnn.summary() 
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = cnn.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting learning rate to: {:.4f}'.format(LEARN[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting learning rate to: {:.4f}'.format(LEARN[bestM]))


Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 62, 47, 32)        320       
                                                                 
 conv2d_25 (Conv2D)          (None, 62, 47, 32)        9248      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 31, 23, 32)       0         
 g2D)                                                            
                                                                 
 batch_normalization_15 (Bat  (None, 31, 23, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_15 (Dropout)        (None, 31, 23, 32)        0         
                                                                 
 conv2d_26 (Conv2D)          (None, 31, 23, 64)      

Highest Mean accuracy acheived on the test set: 0.8405
Highest Mean accuracy acheived by setting learning rate to: 0.0150
Highest Mean accuracy acheived on the test set with mirror set: 0.8641
Highest Mean accuracy acheived by setting learning rate to: 0.0150

In [ ]:
#Find good range of elearning rate for adam
AccMax_te = []
AccMaxMirror_te = []
LEARN = [.0001, .0005, .001, .005]

for lr in LEARN:
  cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=4)
  cnn.summary() 
  rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=2, verbose=1,min_lr=lr)
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.01),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = cnn.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting MIN learning rate to: {:.4f}'.format(LEARN[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting MIN learning rate to: {:.4f}'.format(LEARN[bestM]))


Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_56 (Conv2D)          (None, 62, 47, 32)        320       
                                                                 
 conv2d_57 (Conv2D)          (None, 62, 47, 32)        9248      
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 31, 23, 32)       0         
 g2D)                                                            
                                                                 
 batch_normalization_35 (Bat  (None, 31, 23, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_35 (Dropout)        (None, 31, 23, 32)        0         
                                                                 
 conv2d_58 (Conv2D)          (None, 31, 23, 64)      

Highest Mean accuracy acheived on the test set: 0.8410
Highest Mean accuracy acheived by setting MIN learning rate to: 0.0010
Highest Mean accuracy acheived on the test set with mirror set: 0.8641
Highest Mean accuracy acheived by setting MIN learning rate to: 0.0010

In [ ]:
#Find good range of patience
AccMax_te = []
AccMaxMirror_te = []
patienc = [3, 5, 7, 9]

for p in patienc:
  cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=4)
  cnn.summary() 
  rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=p, verbose=1,min_lr=.0005)
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.01),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = cnn.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting patience to: {:.4f}'.format(patienc[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting patience to: {:.4f}'.format(patienc[bestM]))


Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_88 (Conv2D)          (None, 62, 47, 32)        320       
                                                                 
 conv2d_89 (Conv2D)          (None, 62, 47, 32)        9248      
                                                                 
 max_pooling2d_44 (MaxPoolin  (None, 31, 23, 32)       0         
 g2D)                                                            
                                                                 
 batch_normalization_55 (Bat  (None, 31, 23, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_55 (Dropout)        (None, 31, 23, 32)        0         
                                                                 
 conv2d_90 (Conv2D)          (None, 31, 23, 64)      

Highest Mean accuracy acheived on the test set: 0.8492
Highest Mean accuracy acheived by setting patience to: 4.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.8897
Highest Mean accuracy acheived by setting patience to: 4.0000

In [ ]:
#Find good range of elearning rate for adam
AccMax_te = []
AccMaxMirror_te = []
drop = [.15, .2, .25]
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=4, verbose=1,min_lr=.0005)


for d in drop:
  cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=4, dropout = d)
  cnn.summary() 
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= .01),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = cnn.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=256, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting dropout rate to: {:.4f}'.format(drop[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting dropout rate to: {:.4f}'.format(drop[bestM]))


Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_120 (Conv2D)         (None, 62, 47, 32)        320       
                                                                 
 conv2d_121 (Conv2D)         (None, 62, 47, 32)        9248      
                                                                 
 max_pooling2d_60 (MaxPoolin  (None, 31, 23, 32)       0         
 g2D)                                                            
                                                                 
 batch_normalization_75 (Bat  (None, 31, 23, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_75 (Dropout)        (None, 31, 23, 32)        0         
                                                                 
 conv2d_122 (Conv2D)         (None, 31, 23, 64)      

Highest Mean accuracy acheived on the test set: 0.8533
Highest Mean accuracy acheived by setting dropout rate to: 0.2500
Highest Mean accuracy acheived on the test set with mirror set: 0.9154
Highest Mean accuracy acheived by setting dropout rate to: 0.2500

In [ ]:
#Find good range of elearning rate for adam
AccMax_te = []
AccMaxMirror_te = []
batchS = [115, 120, 125]
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=4, verbose=1,min_lr=.0005)

for b in batchS:
  cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=4, dropout =.25)
  cnn.summary() 
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= .01),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = cnn.fit(
      x_tr, y_tr,
      epochs =  30, 
      batch_size=b, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting batch size  to: {:.4f}'.format(batchS[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting batch size to: {:.4f}'.format(batchS[bestM]))


Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_144 (Conv2D)         (None, 62, 47, 32)        320       
                                                                 
 conv2d_145 (Conv2D)         (None, 62, 47, 32)        9248      
                                                                 
 max_pooling2d_72 (MaxPoolin  (None, 31, 23, 32)       0         
 g2D)                                                            
                                                                 
 batch_normalization_90 (Bat  (None, 31, 23, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_90 (Dropout)        (None, 31, 23, 32)        0         
                                                                 
 conv2d_146 (Conv2D)         (None, 31, 23, 64)      

Highest Mean accuracy acheived on the test set: 0.9262
Highest Mean accuracy acheived by setting batch size  to: 125.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.9487
Highest Mean accuracy acheived by setting batch size to: 125.0000


In [ ]:
#Find good range of elearning rate for adam
AccMax_te = []
AccMaxMirror_te = []
epoc = [ 40, 45, 50]
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=4, verbose=1,min_lr=.0005)

for ep in epoc:
  cnn = vgg_dropout_bn(input_shape= x_tr.shape[1:], blocks=4, dropout =.25)
  cnn.summary() 
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.001),loss="categorical_crossentropy", metrics=["accuracy"])
  all_history = []

  history = cnn.fit(
      x_tr, y_tr,
      epochs =  ep, 
      batch_size= 125, 
      verbose = 1,
      validation_data=(x_te, y_te),
      callbacks=[rop]
  )

  all_history.append(history)
  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))
  y_class = np.argmax(y_te,axis=1)

  # Classify test set
  pred1 = cnn(x_te).numpy()
  # Classify mirrored test set 
  pred2 = cnn(x_te[:,:,::-1]).numpy()
  pred = np.argmax(pred1+pred2,axis=1)

  AccMaxMirror_te.append(accuracy_score(pred,y_class))


bestM = np.argmax(AccMaxMirror_te)
best = np.argmax(AccMax_te)

print('Highest Mean accuracy acheived on the test set: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting epoch to: {:.4f}'.format(epoc[best]))

print('Highest Mean accuracy acheived on the test set with mirror set: {:.4f}'.format(AccMaxMirror_te[bestM]))
print('Highest Mean accuracy acheived by setting epoch to: {:.4f}'.format(epoc[bestM]))


Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_168 (Conv2D)         (None, 62, 47, 32)        320       
                                                                 
 conv2d_169 (Conv2D)         (None, 62, 47, 32)        9248      
                                                                 
 max_pooling2d_84 (MaxPoolin  (None, 31, 23, 32)       0         
 g2D)                                                            
                                                                 
 batch_normalization_105 (Ba  (None, 31, 23, 32)       128       
 tchNormalization)                                               
                                                                 
 dropout_105 (Dropout)       (None, 31, 23, 32)        0         
                                                                 
 conv2d_170 (Conv2D)         (None, 31, 23, 64)      

Test1:

Highest Mean accuracy acheived on the test set: 0.9446
Highest Mean accuracy acheived by setting epoch to: 55.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.9487
Highest Mean accuracy acheived by setting epoch to: 40.0000

Test2:

Highest Mean accuracy acheived on the test set: 0.9446
Highest Mean accuracy acheived by setting epoch to: 50.0000
Highest Mean accuracy acheived on the test set with mirror set: 0.9359
Highest Mean accuracy acheived by setting epoch to: 50.0000

# **Kaggle CelebA dataset**

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls  '/content/drive/MyDrive/CelebA'

image_class2.xlsx  img_align_half_res.zip  list_attr_celeba.xlsx
image_class.xlsx   list_attr_celeba.csv    list_landmarks_align_celeba.csv


In [ ]:
from zipfile import ZipFile
file_name = '/content/drive/MyDrive/CelebA/img_align_half_res.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall('celebA_imgs')
  print('Done')

Done


In [ ]:
#from google.colab import files
#uploaded = files.upload() 

import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/CelebA/image_class2.xlsx')

In [ ]:
import time
import math 
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import distutils

from keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.optimizers import *
from keras.utils.vis_utils import plot_model
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score,mean_squared_error,mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

def prepKaggle(y = 'Gender'):

  datagen = ImageDataGenerator(
      rescale=1./255,
      #shear_range=0.2,
      #zoom_range=0.2,
      validation_split=0.2,
      #horizontal_flip=True
      )
  train_generator = datagen.flow_from_dataframe(df, x_col='filename', y_col=y,target_size=(109, 89),color_mode='rgb',directory='celebA_imgs',seed = 42, batch_size=256, subset='training')
  test_generator = datagen.flow_from_dataframe(df, x_col='filename', y_col=y,target_size=(109, 89),color_mode='rgb',directory='celebA_imgs',seed = 42, batch_size=256, subset='validation')


  return train_generator, test_generator



  
 #crashes 
def prepKaggle2(y = 'Gender'):

  datagen = ImageDataGenerator(
      rescale=1./255,
      #shear_range=0.2,
      #zoom_range=0.2,
      validation_split=0.2,
      #horizontal_flip=True
      )
  train_generator = datagen.flow_from_dataframe(df, x_col='filename', y_col=y,target_size=(109, 89),color_mode='rgb',directory='celebA_imgs',seed = 42, batch_size=162080, subset='training')
  test_generator = datagen.flow_from_dataframe(df, x_col='filename', y_col=y,target_size=(109, 89),color_mode='rgb',directory='celebA_imgs',seed = 42, batch_size=40519,  subset='validation')
  x_tr, y_tr, = next(iter(train_generator))
  x_te, y_te  = next(iter(test_generator))
  return x_tr, y_tr, x_te, y_te
   

##**Dense Neural network Model (MLP) with Kaggle Dataset**

In [ ]:
#View graph of the results 
def plot_results(all_history):
  loss, val_loss, accuracy, val_accuracy = [], [], [], []
  for history in all_history:
    loss += history.history['loss']
    val_loss += history.history['val_loss']
    accuracy += history.history['accuracy']
    val_accuracy += history.history['val_accuracy']

  fig, ax = plt.subplots()
  ax.plot(accuracy,label = 'train')
  ax.plot(val_accuracy,label = 'test')
  ax.set_title('Accuracy')
  ax.legend(loc='lower right')
  fig, ax = plt.subplots()
  ax.plot(loss,label = 'train')
  ax.plot(val_loss,label = 'test')
  ax.set_title('Loss')
  ax.legend(loc='upper right')

#Set up data and lists for MLP models
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=2, verbose=1,min_lr=0.0005)

def dense_model(inputs = (109, 89,3),hidden_1=750, hidden_2=750):
  model = tf.keras.models.Sequential()
  model.add(Flatten(input_shape= inputs ))
  model.add(Dense(hidden_1, activation='relu'))
  model.add(Dense(hidden_2, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  return model

def dense_model3(inputs = (109, 89,3),hidden_1=500, hidden_2=500, hidden_3=500):
  model = tf.keras.models.Sequential()
  model.add(Flatten(input_shape= inputs ))
  model.add(Dense(hidden_1, activation='relu'))
  model.add(Dense(hidden_2, activation='relu'))
  model.add(Dense(hidden_3, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  return model
def dense_model4(inputs = (109, 89,3),hidden_1=500, hidden_2=500, hidden_3=500, hidden_4=500):
  model = tf.keras.models.Sequential()
  model.add(Flatten(input_shape= inputs))
  model.add(Dense(hidden_1, activation='relu'))
  model.add(Dense(hidden_2, activation='relu'))
  model.add(Dense(hidden_3, activation='relu'))
  model.add(Dense(hidden_4, activation='relu'))
  model.add(Dense(12, activation='softmax'))
  return model

In [ ]:
train_generator, test_generator = prepKaggle(y = 'Gender')

dense_network = dense_model(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator,

)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Gender: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_54 (Flatten)        (None, 29103)             0         
                                                                 
 dense_166 (Dense)           (None, 750)               21828000  
                                                                 
 dense_167 (Dense)           (None, 750)               563250    
                                                                 
 dense_168 (Dense)           (None, 2)                 1502      
                                                                 
Total params: 22,392,752
Trainable params: 22,392,752
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
634/634 [=============================

In [ ]:
train_generator, test_generator = prepKaggle(y = 'Gender')

dense_network = dense_model3(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0005),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Gender: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_55 (Flatten)        (None, 29103)             0         
                                                                 
 dense_169 (Dense)           (None, 500)               14552000  
                                                                 
 dense_170 (Dense)           (None, 500)               250500    
                                                                 
 dense_171 (Dense)           (None, 500)               250500    
                                                                 
 dense_172 (Dense)           (None, 2)                 1002      
                                                                 
Total params: 15,054,002
Trainable params: 15,054,002
Non-tr

2 layers
0.9249043583869934


3 layers
0.9229052901268006

In [ ]:
train_generator, test_generator = prepKaggle(y = 'Attractive')

dense_network = dense_model(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0005),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Attractive: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_56 (Flatten)        (None, 29103)             0         
                                                                 
 dense_173 (Dense)           (None, 750)               21828000  
                                                                 
 dense_174 (Dense)           (None, 750)               563250    
                                                                 
 dense_175 (Dense)           (None, 2)                 1502      
                                                                 
Total params: 22,392,752
Trainable params: 22,392,752
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
634/634 [=============================

In [ ]:
train_generator, test_generator = prepKaggle(y = 'Attractive')

dense_network = dense_model3(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0005),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Attractive: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_58"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_57 (Flatten)        (None, 29103)             0         
                                                                 
 dense_176 (Dense)           (None, 500)               14552000  
                                                                 
 dense_177 (Dense)           (None, 500)               250500    
                                                                 
 dense_178 (Dense)           (None, 500)               250500    
                                                                 
 dense_179 (Dense)           (None, 2)                 1002      
                                                                 
Total params: 15,054,002
Trainable params: 15,054,002
Non-tr

attracive
0.74-.75

In [ ]:
train_generator, test_generator = prepKaggle(y = 'Bald')

dense_network = dense_model(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.001),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Bald: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_59"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_58 (Flatten)        (None, 29103)             0         
                                                                 
 dense_180 (Dense)           (None, 750)               21828000  
                                                                 
 dense_181 (Dense)           (None, 750)               563250    
                                                                 
 dense_182 (Dense)           (None, 2)                 1502      
                                                                 
Total params: 22,392,752
Trainable params: 22,392,752
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
634/634 [=============================

In [ ]:
train_generator, test_generator = prepKaggle(y = 'Bald')

dense_network = dense_model3(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0005),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Bald: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_59 (Flatten)        (None, 29103)             0         
                                                                 
 dense_183 (Dense)           (None, 500)               14552000  
                                                                 
 dense_184 (Dense)           (None, 500)               250500    
                                                                 
 dense_185 (Dense)           (None, 500)               250500    
                                                                 
 dense_186 (Dense)           (None, 2)                 1002      
                                                                 
Total params: 15,054,002
Trainable params: 15,054,002
Non-tr

bald


0.977

In [ ]:
train_generator, test_generator = prepKaggle(y= 'Age')

dense_network = dense_model(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0005),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Age: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_61"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_60 (Flatten)        (None, 29103)             0         
                                                                 
 dense_187 (Dense)           (None, 750)               21828000  
                                                                 
 dense_188 (Dense)           (None, 750)               563250    
                                                                 
 dense_189 (Dense)           (None, 2)                 1502      
                                                                 
Total params: 22,392,752
Trainable params: 22,392,752
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
634/634 [=============================

In [ ]:
train_generator, test_generator = prepKaggle(y= 'Age')

dense_network = dense_model3(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.001),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Age: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_62"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_61 (Flatten)        (None, 29103)             0         
                                                                 
 dense_190 (Dense)           (None, 500)               14552000  
                                                                 
 dense_191 (Dense)           (None, 500)               250500    
                                                                 
 dense_192 (Dense)           (None, 500)               250500    
                                                                 
 dense_193 (Dense)           (None, 2)                 1002      
                                                                 
Total params: 15,054,002
Trainable params: 15,054,002
Non-tr

Age

.835



In [ ]:
train_generator, test_generator = prepKaggle(y= 'Heavy_Makeup')

dense_network = dense_model(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.001),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Heavy Makeup detection: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_62 (Flatten)        (None, 29103)             0         
                                                                 
 dense_194 (Dense)           (None, 750)               21828000  
                                                                 
 dense_195 (Dense)           (None, 750)               563250    
                                                                 
 dense_196 (Dense)           (None, 2)                 1502      
                                                                 
Total params: 22,392,752
Trainable params: 22,392,752
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
634/634 [=============================

In [ ]:
train_generator, test_generator = prepKaggle(y= 'Heavy_Makeup')

dense_network = dense_model3(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.001),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for heavy makeup detection: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_64"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_63 (Flatten)        (None, 29103)             0         
                                                                 
 dense_197 (Dense)           (None, 500)               14552000  
                                                                 
 dense_198 (Dense)           (None, 500)               250500    
                                                                 
 dense_199 (Dense)           (None, 500)               250500    
                                                                 
 dense_200 (Dense)           (None, 2)                 1002      
                                                                 
Total params: 15,054,002
Trainable params: 15,054,002
Non-tr

Heavy Makeup

.85-.86%



In [ ]:
train_generator, test_generator = prepKaggle(y= 'Pale_Skin')

dense_network = dense_model(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.001),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for determining Pale Skin {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_65"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_64 (Flatten)        (None, 29103)             0         
                                                                 
 dense_201 (Dense)           (None, 750)               21828000  
                                                                 
 dense_202 (Dense)           (None, 750)               563250    
                                                                 
 dense_203 (Dense)           (None, 2)                 1502      
                                                                 
Total params: 22,392,752
Trainable params: 22,392,752
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
634/634 [=============================

In [ ]:
train_generator, test_generator = prepKaggle(y= 'Pale_Skin')

dense_network = dense_model3(inputs= train_generator.image_shape)
dense_network.summary() 

#learn rate usually at .001
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.001),loss="binary_crossentropy", metrics=["accuracy"])

all_history = []
history = dense_network.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data = test_generator
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for determining Pale Skin: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_66"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_65 (Flatten)        (None, 29103)             0         
                                                                 
 dense_204 (Dense)           (None, 500)               14552000  
                                                                 
 dense_205 (Dense)           (None, 500)               250500    
                                                                 
 dense_206 (Dense)           (None, 500)               250500    
                                                                 
 dense_207 (Dense)           (None, 2)                 1002      
                                                                 
Total params: 15,054,002
Trainable params: 15,054,002
Non-tr

Pale_Skin

.96

##**CNN Model with Kaggle Dataset**

In [ ]:
def vgg_dropout_bn(input_shape=(109, 89,3),blocks=3,branching_factor=2,first_layer_filters=32,dropout=0.15,dense_layer=500):
  model = tf.keras.models.Sequential()
  f = first_layer_filters
  for i in range(blocks):
    if i==0:
      model.add(Conv2D(f, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation="relu"))
    else:
      model.add(Conv2D(f, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(f, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    #model.add(BatchNormalization())
    model.add(Dropout(dropout))
    f = int(f*branching_factor)
  model.add(Flatten())
  if dense_layer>0:
    model.add(Dense(dense_layer,kernel_regularizer=l2(0.0001), activation='relu'))
    model.add(Dense(dense_layer,kernel_regularizer=l2(0.0001), activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dropout(dropout))
  model.add(Dense(2,activation='softmax'))
  #model.add(Activation('sigmoid'))
  return model



In [ ]:
train_generator, test_generator = prepKaggle(y = 'Gender')

cnn = vgg_dropout_bn(input_shape = train_generator.image_shape, blocks=4)
cnn.summary() 
cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy", metrics=["accuracy"])
all_history = []
 
history = cnn.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data= test_generator,

)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Gender: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_116"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_396 (Conv2D)         (None, 109, 89, 32)       896       
                                                                 
 conv2d_397 (Conv2D)         (None, 109, 89, 32)       9248      
                                                                 
 max_pooling2d_198 (MaxPooli  (None, 54, 44, 32)       0         
 ng2D)                                                           
                                                                 
 dropout_248 (Dropout)       (None, 54, 44, 32)        0         
                                                                 
 conv2d_398 (Conv2D)         (None, 54, 44, 64)        18496     
                                                           

Gender

.96-.97


In [ ]:
LEARN = [.00005, .0001, .0005, .001]
AccMax_te = []
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=2, verbose=1,min_lr=0.0005)

for lr in LEARN:
  train_generator, test_generator = prepKaggle(y = 'Attractive')
  cnn = vgg_dropout_bn(input_shape = train_generator.image_shape, blocks=3)
  cnn.summary() 
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= lr),loss="binary_crossentropy", metrics=["accuracy"])
  all_history = []
  
  history = cnn.fit(
      train_generator,
      epochs =  5, 
      verbose = 1,
      validation_data= test_generator,
      callbacks=[rop]
  )

  all_history.append(history)

  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))

best = np.argmax(AccMax_te)
print('Highest Mean accuracy acheived on the test set for Attractive: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting learning rate to: {:.4f}'.format(LEARN[best]))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_51"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_314 (Conv2D)         (None, 109, 89, 32)       896       
                                                                 
 conv2d_315 (Conv2D)         (None, 109, 89, 32)       9248      
                                                                 
 max_pooling2d_157 (MaxPooli  (None, 54, 44, 32)       0         
 ng2D)                                                           
                                                                 
 dropout_196 (Dropout)       (None, 54, 44, 32)        0         
                                                                 
 conv2d_316 (Conv2D)         (None, 54, 44, 64)        18496     
                                                            

In [ ]:
LEARN = [.00005, .0001, .0005, .001]
AccMax_te = []
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=2, verbose=1,min_lr=lr)

for lr in LEARN:
  train_generator, test_generator = prepKaggle(y = 'Attractive')
  cnn = vgg_dropout_bn(input_shape = train_generator.image_shape, blocks=3)
  cnn.summary() 
  cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= lr),loss="binary_crossentropy", metrics=["accuracy"])
  all_history = []
  
  history = cnn.fit(
      train_generator,
      epochs =  5, 
      verbose = 1,
      validation_data= test_generator,
      callbacks=[rop]
  )

  all_history.append(history)

  AccMax_te.append(np.mean(history.history['val_accuracy'][-5:]))

best = np.argmax(AccMax_te)
print('Highest Mean accuracy acheived on the test set for Attractive: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting MIN learning rate to: {:.4f}'.format(LEARN[best]))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_51"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_314 (Conv2D)         (None, 109, 89, 32)       896       
                                                                 
 conv2d_315 (Conv2D)         (None, 109, 89, 32)       9248      
                                                                 
 max_pooling2d_157 (MaxPooli  (None, 54, 44, 32)       0         
 ng2D)                                                           
                                                                 
 dropout_196 (Dropout)       (None, 54, 44, 32)        0         
                                                                 
 conv2d_316 (Conv2D)         (None, 54, 44, 64)        18496     
                                                            

In [ ]:
bloc = [3,4]
AccMax_te = []
rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=2, verbose=1,min_lr=0.0005)

for b in bloc:
  train_generator, test_generator = prepKaggle(y = 'Attractive')
  cnn = vgg_dropout_bn(input_shape = train_generator.image_shape, blocks=b)
  cnn.summary() 
  cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy", metrics=["accuracy"])
  all_history = []
  
  history = cnn.fit(
      train_generator,
      epochs =  5, 
      verbose = 1,
      validation_data= test_generator,
      callbacks=[rop]
  )

  all_history.append(history)

  AccMax_te.append(np.mean(history.history['val_accuracy'][-3:]))

best = np.argmax(AccMax_te)
print('Highest Mean accuracy acheived on the test set for Attractive: {:.4f}'.format(AccMax_te[best]))
print('Highest Mean accuracy acheived by setting Number of Blocks to: {:.4f}'.format(bloc[best]))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_266 (Conv2D)         (None, 109, 89, 32)       896       
                                                                 
 conv2d_267 (Conv2D)         (None, 109, 89, 32)       9248      
                                                                 
 max_pooling2d_133 (MaxPooli  (None, 54, 44, 32)       0         
 ng2D)                                                           
                                                                 
 dropout_166 (Dropout)       (None, 54, 44, 32)        0         
                                                                 
 conv2d_268 (Conv2D)         (None, 54, 44, 64)        18496     
                                                            

In [ ]:

rop = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience=2, verbose=1,min_lr=0.0005)

train_generator, test_generator = prepKaggle(y = 'Attractive')
cnn = vgg_dropout_bn(input_shape = train_generator.image_shape, blocks=3)
cnn.summary() 
cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy", metrics=["accuracy"])
all_history = []

history = cnn.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data= test_generator,
    #callbacks=[rop]
  )

all_history.append(history)

print('Highest Mean accuracy acheived on the test set for Attractive: {:.4f}'.format(np.mean(history.history['val_accuracy'][-3:])))


Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_121"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_436 (Conv2D)         (None, 109, 89, 32)       896       
                                                                 
 conv2d_437 (Conv2D)         (None, 109, 89, 32)       9248      
                                                                 
 max_pooling2d_218 (MaxPooli  (None, 54, 44, 32)       0         
 ng2D)                                                           
                                                                 
 dropout_270 (Dropout)       (None, 54, 44, 32)        0         
                                                                 
 conv2d_438 (Conv2D)         (None, 54, 44, 64)        18496     
                                                           

TypeError: ignored

Attractive

.79 - .80



In [ ]:
train_generator, test_generator = prepKaggle(y = 'Bald')
cnn = vgg_dropout_bn(input_shape = train_generator.image_shape, blocks=4)
cnn.summary() 
cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy", metrics=["accuracy"])
all_history = []
 
history = cnn.fit(
    train_generator,
    epochs =  5, 
    verbose = 1,
    validation_data= test_generator,
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Bald: {:.3f}'.format(np.mean(history.history['val_accuracy'][-3:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_114"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_380 (Conv2D)         (None, 109, 89, 32)       896       
                                                                 
 conv2d_381 (Conv2D)         (None, 109, 89, 32)       9248      
                                                                 
 max_pooling2d_190 (MaxPooli  (None, 54, 44, 32)       0         
 ng2D)                                                           
                                                                 
 batch_normalization_75 (Bat  (None, 54, 44, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_238 (Dropout)       (None, 54, 44, 32)        0   

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/5


InvalidArgumentError: ignored

bald

0.9835385878880819


In [ ]:
train_generator, test_generator = prepKaggle(y = 'Age')
cnn = vgg_dropout_bn(input_shape = train_generator.image_shape, blocks=4)
cnn.summary() 
cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy", metrics=["accuracy"])
all_history = []
 
history = cnn.fit(
    train_generator,
    epochs =  3, 
    verbose = 1,
    validation_data= test_generator,
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Age: {:.3f}'.format(np.mean(history.history['val_accuracy'][-5:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_48 (Conv2D)          (None, 109, 89, 32)       896       
                                                                 
 conv2d_49 (Conv2D)          (None, 109, 89, 32)       9248      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 54, 44, 32)       0         
 g2D)                                                            
                                                                 
 dropout_30 (Dropout)        (None, 54, 44, 32)        0         
                                                                 
 conv2d_50 (Conv2D)          (None, 54, 44, 64)        18496     
                                                             

Age

0.866227368513743

In [ ]:
train_generator, test_generator = prepKaggle(y = 'Heavy_Makeup')
cnn = vgg_dropout_bn(input_shape = train_generator.image_shape, blocks=4)
cnn.summary() 
cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy", metrics=["accuracy"])
all_history = []
 
history = cnn.fit(
    train_generator,
    epochs =  3, 
    verbose = 1,
    validation_data= test_generator,
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Heavy Makeup detection: {:.3f}'.format(np.mean(history.history['val_accuracy'][-5:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 109, 89, 32)       896       
                                                                 
 conv2d_9 (Conv2D)           (None, 109, 89, 32)       9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 44, 32)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 54, 44, 32)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 54, 44, 64)        18496     
                                                             

Heavy Makeup

0.9013302326202393

In [ ]:
train_generator, test_generator = prepKaggle(y = 'Pale_Skin')
cnn = vgg_dropout_bn(input_shape = train_generator.image_shape, blocks=4)
cnn.summary() 
cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy", metrics=["accuracy"])
all_history = []
 
history = cnn.fit(
    train_generator,
    epochs =  3, 
    verbose = 1,
    validation_data= test_generator,
)

all_history.append(history)
print('Highest Mean accuracy acheived on the test set for Pale_Skin: {:.3f}'.format(np.mean(history.history['val_accuracy'][-5:])))

Found 162080 validated image filenames belonging to 2 classes.
Found 40519 validated image filenames belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 109, 89, 32)       896       
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 89, 32)       9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 54, 44, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 54, 44, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 54, 44, 64)        18496     
                                                               

Pale_Skin

0.9651932915051779